# Retail Analysis with MongoDB

In [11]:
import yaml
from pymongo import  MongoClient
import pandas as pd

# Load config.yaml
with open("../config.yaml", 'r') as f:
    cfg = yaml.safe_load(f)

client = MongoClient(cfg["mongodb"]["uri"])
db = client[cfg["mongodb"]["database"]]

In [29]:
# Load curated data
sales_col = db["daily_sales"]
customers_col = db["customers_curated"]
stock_col = db["stock"]

print("Collections:", db.list_collection_names())

Collections: ['customers_curated', 'daily_sales', 'stock']


In [25]:
sales_col.count_documents({}) # Verify if the data is loaded

36

In [26]:
customers_col.count_documents({})

4

In [30]:
stock_col.count_documents({})

144

## Top 10 SKUs by Quantity sold

In [14]:
pipeline = [
    {
        "$group": {
            "_id": "$sku_id",
            "total_quantity": {"$sum": "$total_qty"},
            "total_sales": {"$sum": "$total_sales"},
        }
    },
    {"$sort": {"total_quantity": -1}},
    {"$limit":10}
]
top_skus =list(sales_col.aggregate(pipeline)) # MongoDB outputs json like files, so we will be using pandas as it renders better tables. If you want better output, you can make connection with mongo Compass , GUI for MongoDB and just do the same query.

df_top_skus = pd.DataFrame(top_skus)
df_top_skus.rename(columns={"_id": "sku_id"}, inplace=True)
df_top_skus


,sku_id,total_quantity,total_sales
0,P001,2961,297894.61
1,P002,2259,448804.17
2,P003,2196,649871.84


I had only 3 sku id in  my dataset.

## Daily sales trend for specific SKU

In [15]:
sku_id = "P002"  # Example SKU
pipeline = [
    {"$match": {"sku_id": sku_id}},    # Filter for this SKU
    {
        "$group": {
            "_id": "$date",            # Group by date
            "daily_quantity": {"$sum": "$total_qty"},  # Sum quantity per day
            "daily_sales": {"$sum": "$total_sales"}    # Sum revenue per day
        }
    },
    {"$sort": {"_id": 1}}  # Sort by date ascending
]

daily_trend = list(sales_col.aggregate(pipeline))
df_daily_trend = pd.DataFrame(daily_trend)
df_daily_trend.rename(columns={"_id": "date"}, inplace=True)
df_daily_trend

,date,daily_quantity,daily_sales
0,2025-08-29 14:00:00,442,87230.34
1,2025-08-30 14:00:00,426,85029.13
2,2025-08-31 14:00:00,223,45128.50
3,2025-09-01 14:00:00,330,63700.89
4,2025-09-02 14:00:00,406,81413.55
5,2025-09-03 14:00:00,432,86301.76


## High-Value customers that are on likely to leave

In [17]:
pipeline = [
    {
        "$match": {
            "$or": [
                {"monetary_score": {"$gte": 3}},
                {"frequency_score": {"$gte": 3}}
            ],
            "recency_score": {"$lte": 2}   # haven't purchased recently
        }
    },
    {"$sort": {"monetary": -1, "frequency": -1, "recency": 1}},  # prioritize high spenders
    {"$limit": 10}  # top 10
]

at_risk_customers = list(customers_col.aggregate(pipeline))
df_at_risk_customers = pd.DataFrame(at_risk_customers)
df_at_risk_customers.rename(columns={"_id": "mongo_id"}, inplace=True)
df_at_risk_customers


,mongo_id,customer_id,last_purchase_date,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_total_score
0,68ce9f781b0dbb6d48724d94,C002,2025-08-25 14:00:00,25,27,5292.0,2,3,4,9
1,68ce9f781b0dbb6d48724d91,C003,2025-08-22 14:00:00,28,26,2574.0,1,2,3,6


## High-frequency low-spend customers
Targeting customers for upselling or cross-selling campaigns

In [20]:
pipeline = [
    {
        "$match": {
            "frequency_score": {"$gte": 3},   # frequent buyers
            "monetary_score": {"$lte": 2}     # low spend
        }
    },
    {
        "$addFields": {
            "avg_order_value": {
                "$divide": ["$monetary", "$frequency"]  # average spend per order
            }
        }
    },
    {"$sort": {"avg_order_value": 1}},  # lowest avg order value first
    {"$limit": 10}                      # top 10 low-value frequent customers
]

low_value_frequent = list(customers_col.aggregate(pipeline))
df_low_value_frequent = pd.DataFrame(low_value_frequent)
df_low_value_frequent.rename(columns={"_id": "mongo_id"}, inplace=True)
df_low_value_frequent


,mongo_id,customer_id,last_purchase_date,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_total_score,avg_order_value
0,68ce9f781b0dbb6d48724d92,C004,2025-08-27 14:00:00,23,37,2331.0,4,4,2,10,63.0


## Stores with Most At-Risk Skus

In [36]:
pipeline = [
    {
        "$match": {
            "stock_out_signal": 1
        }
    },
    {
        "$group": {
            "_id": "$store_id",
            "num_at_risk_skus": {"$sum": 1},
            "avg_stock_qty": {"$avg": "$on_stock"}
        }
    },
    {"$sort": {"num_at_risk_skus": -1}},
    {"$limit": 5}
]

stores_at_risk = list(stock_col.aggregate(pipeline))
df_stores_at_risk = pd.DataFrame(stores_at_risk)
df_stores_at_risk.rename(columns={"_id": "store_id"}, inplace=True)
df_stores_at_risk



,store_id,num_at_risk_skus,avg_stock_qty
0,X001,56,103.285714
1,X002,50,83.240000


## Critical SKUs by Store
It shows which store is most critical for restocking

In [39]:
pipeline = [
    # Group by store and SKU
    {
        "$group": {
            "_id": {"store_id": "$store_id", "sku_id": "$sku_id"},
            "total_on_stock": {"$sum": "$on_stock"},
            "total_moving_avg": {"$sum": "$moving_avg_qty"}
        }
    },
    # Compute stock-to-demand ratio
    {
        "$addFields": {
            "stock_to_avg_sales_ratio": {
                "$divide": ["$total_on_stock", "$total_moving_avg"]
            }
        }
    },
    # Filter SKUs at risk
    {
        "$match": {
            "stock_to_avg_sales_ratio": {"$lte": 1}
        }
    },
    # Sort by ratio ascending (most urgent first)
    {"$sort": {"stock_to_avg_sales_ratio": 1}},
    {"$limit": 20}  # Top 20 for visibility
]

critical_skus_store = list(stock_col.aggregate(pipeline))

# Convert to DataFrame for better visualization
df_critical_skus_store = pd.DataFrame(critical_skus_store)
df_critical_skus_store = pd.concat([df_critical_skus_store.drop(columns=["_id"]),
                                    df_critical_skus_store["_id"].apply(pd.Series)], axis=1)

df_critical_skus_store


,total_on_stock,total_moving_avg,stock_to_avg_sales_ratio,store_id,sku_id
0,2968,5539.826667,0.535757,X001,P001
1,3150,5682.563333,0.554327,X001,P002
2,3270,5618.893333,0.581965,X002,P003
3,3096,4682.166667,0.661232,X002,P001
4,2956,4192.506667,0.705067,X002,P002
5,3700,4117.603333,0.898581,X001,P003
